<h1 style="color":"blue"> Goodreads vs. NY Times Bestsellers </h1>

PROPOSAL:

Datasets:
New York Time Bestsellers: https://www.kaggle.com/dhruvildave/new-york-times-best-sellers?select=bestsellers.csv
Goodreads:
https://www.kaggle.com/jealousleopard/goodreadsbooks

We will be extracting CSV data from the NYT Bestsellers dataset and a dataset from Goodreads and combining into a single database using ISBN-13 as a primary key. 

We will transform and clean the data in Pandas, removing any duplicates within the same dataset and verifying character length fields prior to loading into the database.

The result in a SQL relational database. Our datasets are static and will not be updated.

The data between NYTime and Goodreads will allow a user to view reader ratings and popularity, book length, the price of the book, length of time on the bestseller list, and a link to purchase. 

We will be creating 6 tables in a Books_DB database from the Goodreads and NYTimes data.
These tables will contain information on titles that are common between the datasets.
Our tables will be:
- Publisher Data
- Author Data
- Bestseller List Data
- Reader Review Data
- Sales Data
- Book Data

In [1]:
# Importing necessary modules
import pandas as pd
from sqlalchemy import create_engine


In [2]:
# Loading NYTimes bestsellers CSV as DataFrame
bestsellers_file = "Resources/bestsellers.csv"
bestsellers_df = pd.read_csv(bestsellers_file)
bestsellers_df.dtypes

published_date         object
list_name              object
list_name_encoded      object
rank                    int64
isbn13                 object
isbn10                 object
title                  object
author                 object
description            object
amazon_product_url     object
price                 float64
weeks_on_list           int64
dtype: object

In [3]:
# Loading Goodreads CSV as DataFrame
books_file = "Resources/book.csv"
books_df = pd.read_csv(books_file, sep=",")
books_df.dtypes


bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                float64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [4]:
# Renaming ISBN column to ISBN10 to match NYTimes DF, as we will be joining on this field
books_clean = books_df.rename(columns = {'isbn': 'isbn10'})
books_clean.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn10                 object
isbn13                float64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [5]:
# Creating a DF of publisher data for the Publisher Tabler
publisher_df = books_df[["publisher"]].copy()
publisher_df.head()

,publisher
0,Scholastic Inc.
1,Scholastic Inc.
2,Scholastic
3,Scholastic Inc.
4,Scholastic


In [6]:
# Create a unique DataFrame of publishers with an unique ID
publisher_unique = pd.DataFrame(publisher_df.drop_duplicates('publisher').reset_index())
publisher_unique.head()

,index,publisher
0,0,Scholastic Inc.
1,2,Scholastic
2,5,Nimble Books
3,7,Gramercy Books
4,8,Del Rey Books


In [6]:
# Renaming columns
publisher_rename = publisher_unique.rename(columns = { "index": "id", "publisher": "publisher_name"})
publisher_rename

,id,publisher_name
0,0,Scholastic Inc.
1,2,Scholastic
2,5,Nimble Books
3,7,Gramercy Books
4,8,Del Rey Books
...,...,...
2285,11094,Folio histoire
2286,11101,Alfaguara Infantil
2287,11107,Suma
2288,11109,Panamericana Editorial


# Why do a merge?

We merged the data from the Goodreads and NYTimes Bestseller list in order to create a database containing books that exist on both lists. Our "Books" table contains information from both datasets.

This will allow users to view books that have bestseller and goodreads information to analyze reader reviews vs critical acclaim.

In [74]:
# Merging the cleaned Goodreads dataframe and the Bestsellers dataframe on the ISBN10 data that exists in both datasets.
books_joined_df = pd.merge(books_clean, bestsellers_df, how="inner", on="isbn10")
books_joined_df

,bookID,title_x,authors,average_rating,isbn10,isbn13_x,language_code,num_pages,ratings_count,text_reviews_count,...,list_name,list_name_encoded,rank,isbn13_y,title_y,author,description,amazon_product_url,price,weeks_on_list
0,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.780440e+12,eng,870,2153167,29221,...,Series Books,series-books,2,9.78044E+12,HARRY POTTER,JK Rowling,A wizard hones his skills while fighting evil.,https://www.amazon.com/Harry-Potter-And-Order-...,0.0,264
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.780440e+12,eng,870,2153167,29221,...,Series Books,series-books,3,9.78044E+12,HARRY POTTER,JK Rowling,A wizard hones his conjuring skills in the ser...,https://www.amazon.com/Harry-Potter-And-Order-...,0.0,330
2,865,The Alchemist,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.86,61122416,9.780060e+12,eng,197,1631221,55843,...,Trade Fiction Paperback,trade-fiction-paperback,4,9.78006E+12,THE ALCHEMIST,Paulo Coelho,"In this fable, a Spanish shepherd boy ventures...",https://www.amazon.com/Alchemist-Paulo-Coelho/...,0.0,255
3,865,The Alchemist,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.86,61122416,9.780060e+12,eng,197,1631221,55843,...,Trade Fiction Paperback,trade-fiction-paperback,5,9.78006E+12,THE ALCHEMIST,Paulo Coelho,"In this fable, a Spanish shepherd boy ventures...",https://www.amazon.com/Alchemist-Paulo-Coelho/...,0.0,263
4,865,The Alchemist,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.86,61122416,9.780060e+12,eng,197,1631221,55843,...,Trade Fiction Paperback,trade-fiction-paperback,5,9.78006E+12,THE ALCHEMIST,Paulo Coelho,"In this fable, a Spanish shepherd boy ventures...",https://www.amazon.com/Alchemist-Paulo-Coelho/...,0.0,264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,44187,The Sea of Monsters (Percy Jackson and the Oly...,Rick Riordan,4.24,1423103343,9.781420e+12,eng,280,10551,1205,...,Series Books,series-books,2,9.78142E+12,PERCY JACKSON & THE OLYMPIANS,Rick Riordan,Children of the gods battle mythological monst...,https://www.amazon.com/Titans-Curse-Percy-Jack...,0.0,142
1047,44187,The Sea of Monsters (Percy Jackson and the Oly...,Rick Riordan,4.24,1423103343,9.781420e+12,eng,280,10551,1205,...,Series Books,series-books,2,9.78142E+12,PERCY JACKSON & THE OLYMPIANS,Rick Riordan,Battling mythological monsters.,https://www.amazon.com/Titans-Curse-Percy-Jack...,0.0,146
1048,44187,The Sea of Monsters (Percy Jackson and the Oly...,Rick Riordan,4.24,1423103343,9.781420e+12,eng,280,10551,1205,...,Series Books,series-books,1,9.78142E+12,PERCY JACKSON & THE OLYMPIANS,Rick Riordan,Children of the gods battle mythological monst...,https://www.amazon.com/Titans-Curse-Percy-Jack...,0.0,148
1049,44187,The Sea of Monsters (Percy Jackson and the Oly...,Rick Riordan,4.24,1423103343,9.781420e+12,eng,280,10551,1205,...,Series Books,series-books,3,9.78142E+12,PERCY JACKSON & THE OLYMPIANS,Rick Riordan,Children of the gods fight monsters of myth.,https://www.amazon.com/Titans-Curse-Percy-Jack...,0.0,175


In [75]:
# Removing duplicate ISBNs from merged dataframe
books_dropped = books_joined_df.drop_duplicates('isbn10')
len(books_dropped)

29

In [76]:
books_dropped

,bookID,title_x,authors,average_rating,isbn10,isbn13_x,language_code,num_pages,ratings_count,text_reviews_count,...,list_name,list_name_encoded,rank,isbn13_y,title_y,author,description,amazon_product_url,price,weeks_on_list
0,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.780440e+12,eng,870,2153167,29221,...,Series Books,series-books,2,9.78044E+12,HARRY POTTER,JK Rowling,A wizard hones his skills while fighting evil.,https://www.amazon.com/Harry-Potter-And-Order-...,0.00,264
2,865,The Alchemist,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.86,61122416,9.780060e+12,eng,197,1631221,55843,...,Trade Fiction Paperback,trade-fiction-paperback,4,9.78006E+12,THE ALCHEMIST,Paulo Coelho,"In this fable, a Spanish shepherd boy ventures...",https://www.amazon.com/Alchemist-Paulo-Coelho/...,0.00,255
56,1845,Into the Wild,Jon Krakauer,3.98,385486804,9.780390e+12,eng,207,800349,18198,...,Expeditions Disasters and Adventures,expeditions-disasters-and-adventures,1,9.78039E+12,INTO THE WILD,Jon Krakauer,A man's obsession with the wilderness ends in ...,https://www.amazon.com/Into-Wild-Jon-Krakauer-...,0.00,0
155,1898,Into Thin Air: A Personal Account of the Mount...,Jon Krakauer,4.17,385494785,9.780390e+12,eng,368,351406,11701,...,Expeditions Disasters and Adventures,expeditions-disasters-and-adventures,3,9.78039E+12,INTO THIN AIR,Jon Krakauer,A chronicle of the 1996 Mount Everest expediti...,https://www.amazon.com/Into-Thin-Air-Personal-...,0.00,0
160,2199,Team of Rivals: The Political Genius of Abraha...,Doris Kearns Goodwin,4.28,743270754,9.780740e+12,eng,916,133840,6118,...,Paperback Nonfiction,paperback-nonfiction,5,9.78074E+12,TEAM OF RIVALS,Doris Kearns Goodwin,The political genius of Abraham Lincoln.,https://www.amazon.com/Team-Rivals-Political-A...,21.00,38
181,2612,The Tipping Point: How Little Things Can Make ...,Malcolm Gladwell,3.97,316346624,9.780320e+12,eng,301,633037,11898,...,Paperback Business Books,paperback-business-books,5,9.78032E+12,TIPPING POINT,Malcolm Gladwell,How and why certain products and ideas become ...,https://www.amazon.com/The-Tipping-Point-Littl...,0.00,0
272,2767,A People's History of the United States,Howard Zinn,4.08,60838655,9.780060e+12,eng,729,167321,4711,...,Paperback Nonfiction,paperback-nonfiction,4,9.78006E+12,A PEOPLE'S HISTORY OF THE UNITED STATES: 1492 ...,Howard Zinn,"An account from the point of view of women, Af...",https://www.amazon.com/Peoples-History-United-...,18.95,1
274,3869,A Brief History of Time,Stephen Hawking,4.17,553380168,9.780550e+12,eng,212,239652,5860,...,Paperback Nonfiction,paperback-nonfiction,5,9.78055E+12,A BRIEF HISTORY OF TIME,Stephen Hawking,The British cosmologist reviews efforts to cre...,https://www.amazon.com/Brief-History-Time-Step...,0.00,2
308,4588,Extremely Loud and Incredibly Close,Jonathan Safran Foer,3.98,618711651,9.780620e+12,eng,326,335477,19241,...,Trade Fiction Paperback,trade-fiction-paperback,5,9.78062E+12,EXTREMELY LOUD AND INCREDIBLY CLOSE,Jonathan Safran Foer,A precocious boy goes on a scavenger hunt thro...,https://www.amazon.com/Extremely-Loud-Incredib...,14.95,2
320,6149,Beloved,Toni Morrison,3.83,1400033411,9.781400e+12,eng,324,282045,9004,...,Trade Fiction Paperback,trade-fiction-paperback,2,9.7814E+12,BELOVED,Toni Morrison,Winner of the 1988 Pulitzer Prize for fiction....,https://www.amazon.com/Beloved-Toni-Morrison/d...,0.00,1


In [77]:
books_joined_df.columns


Index(['bookID', 'title_x', 'authors', 'average_rating', 'isbn10', 'isbn13_x',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher', 'published_date', 'list_name',
       'list_name_encoded', 'rank', 'isbn13_y', 'title_y', 'author',
       'description', 'amazon_product_url', 'price', 'weeks_on_list'],
      dtype='object')

In [78]:
# Creating a dataframe with only needed columns
books_db_df = books_joined_df[['isbn10', 'title_x', 'description', 'language_code', 'published_date', '  num_pages', 'author', 'publisher']]

In [79]:
# Renaming columns for readability
books_renamed = books_db_df.rename(columns = {'  num_pages': 'num_pages', 'title_x': 'title', 'language_code': 'language'})
books_renamed.columns

Index(['isbn10', 'title', 'description', 'language', 'published_date',
       'num_pages', 'author', 'publisher'],
      dtype='object')

In [11]:
# books_df.rename(columns={'isbn': 'isbn10'}, inplace=True)

In [80]:
#selecting desired columns for sales and review dataframes
col_sales= ['isbn10','price', 'amazon_product_url']
col_review= ['isbn10','average_rating','ratings_count','text_reviews_count']
#creating a dataframe with the columns
df_review = books_df[col_review]
sales_df = bestsellers_df[col_sales]
df_sales.dtypes
df_sales = sales_df.dropna(subset = ['isbn10'])

In [48]:
# selecting desired columns for bestseller list dataframe
best_sellers = bestsellers_df[['isbn10', 'list_name', 'rank', 'weeks_on_list']]

# dropping rows with no ISBN
best_sellers_df = best_sellers.dropna(subset = ['isbn10'])
best_sellers_df.head()

,isbn10,list_name,rank,weeks_on_list
0,316036242,Chapter Books,1,1
1,439023483,Chapter Books,2,67
2,439023491,Chapter Books,3,16
3,763644102,Chapter Books,4,15
4,385738935,Chapter Books,5,2


In [14]:
# selecting desired columns for author dataframe
author_df = books_df[['authors']]
author_df.head()

,authors
0,J.K. Rowling/Mary GrandPré
1,J.K. Rowling/Mary GrandPré
2,J.K. Rowling
3,J.K. Rowling/Mary GrandPré
4,J.K. Rowling/Mary GrandPré


In [15]:
# Create a unique DataFrame of authors with an unique ID

author_unique = pd.DataFrame(author_df.drop_duplicates('authors').reset_index())
author_unique.head()

,index,authors
0,0,J.K. Rowling/Mary GrandPré
1,2,J.K. Rowling
2,5,W. Frederick Zimmerman
3,7,Douglas Adams
4,10,Douglas Adams/Stephen Fry


In [16]:
# Renaming columns for readability

author_rename = author_unique.rename(columns = { "index": "id", "authors":"author"})

In [59]:
# Joining on author name in order to add author_id into books dataframe

books_authorid_df = pd.merge(books_renamed, author_rename, how="outer", on="author")
books_authorid_df
books_author = books_authorid_df.rename(columns={"id":"author_id", "publisher":"publisher_name"})
books_author

,isbn10,title,description,language,published_date,num_pages,author,publisher_name,author_id
0,439358078,Harry Potter and the Order of the Phoenix (Har...,A wizard hones his skills while fighting evil.,eng,8/21/2011,870.0,JK Rowling,Scholastic Inc.,NaN
1,439358078,Harry Potter and the Order of the Phoenix (Har...,A wizard hones his conjuring skills in the ser...,eng,8/2/2015,870.0,JK Rowling,Scholastic Inc.,NaN
2,61122416,The Alchemist,"In this fable, a Spanish shepherd boy ventures...",eng,6/16/2013,197.0,Paulo Coelho,HarperCollins,426.0
3,61122416,The Alchemist,"In this fable, a Spanish shepherd boy ventures...",eng,8/11/2013,197.0,Paulo Coelho,HarperCollins,426.0
4,61122416,The Alchemist,"In this fable, a Spanish shepherd boy ventures...",eng,8/18/2013,197.0,Paulo Coelho,HarperCollins,426.0
...,...,...,...,...,...,...,...,...,...
7664,NaN,NaN,NaN,NaN,NaN,NaN,Mario Vargas Llosa,NaN,11107.0
7665,NaN,NaN,NaN,NaN,NaN,NaN,C.S. Lewis/Margarita E. Valdes/Gemma Gallart/P...,NaN,11110.0
7666,NaN,NaN,NaN,NaN,NaN,NaN,C.S. Lewis/Ana Falcão Bastos,NaN,11111.0
7667,NaN,NaN,NaN,NaN,NaN,NaN,C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,NaN,11112.0


In [60]:
# Joining on publisher name in order to add publisher_id into books dataframe

books_publisherid = pd.merge(books_author, publisher_rename, how="inner", on="publisher_name")

books_ids = books_publisherid.rename(columns={"id":"publisher_id"})
books_ids

,isbn10,title,description,language,published_date,num_pages,author,publisher_name,author_id,publisher_id
0,439358078,Harry Potter and the Order of the Phoenix (Har...,A wizard hones his skills while fighting evil.,eng,8/21/2011,870.0,JK Rowling,Scholastic Inc.,NaN,0
1,439358078,Harry Potter and the Order of the Phoenix (Har...,A wizard hones his conjuring skills in the ser...,eng,8/2/2015,870.0,JK Rowling,Scholastic Inc.,NaN,0
2,61122416,The Alchemist,"In this fable, a Spanish shepherd boy ventures...",eng,6/16/2013,197.0,Paulo Coelho,HarperCollins,426.0,284
3,61122416,The Alchemist,"In this fable, a Spanish shepherd boy ventures...",eng,8/11/2013,197.0,Paulo Coelho,HarperCollins,426.0,284
4,61122416,The Alchemist,"In this fable, a Spanish shepherd boy ventures...",eng,8/18/2013,197.0,Paulo Coelho,HarperCollins,426.0,284
...,...,...,...,...,...,...,...,...,...,...
1046,1565125606,Water for Elephants,Distraught after the death of his parents in a...,eng,7/24/2011,335.0,Sara Gruen,Algonquin Books,10700.0,1318
1047,1565125606,Water for Elephants,Distraught after the death of his parents in a...,eng,7/31/2011,335.0,Sara Gruen,Algonquin Books,10700.0,1318
1048,1565125606,Water for Elephants,Distraught after the death of his parents in a...,eng,8/7/2011,335.0,Sara Gruen,Algonquin Books,10700.0,1318
1049,1565125606,Water for Elephants,Distraught after the death of his parents in a...,eng,8/14/2011,335.0,Sara Gruen,Algonquin Books,10700.0,1318


In [62]:
#Dropping duplicates ISBNs after merge
books_semifinal = books_ids.drop_duplicates('isbn10')
len(books_semifinal)

29

# ...and this is when we discovered...
The Goodreads dataset of 11k unique rows and the NYTimes Bestsellers dataset of 8.1k unique rows, only 29 books existed on both lists.
🤯🤯🤯
The Goodreads dataset that we are using is incomplete, and therefore most of the books on the NYTiems Bestseller list did not exist in the Goodreads dataset.

In [82]:
# Define our dataframe that will be used to create the Books table in our database, with only the necessary columns.
books_final = books_semifinal[['isbn10', 'title', 'description', 'language', 'published_date', 'publisher_id', 'num_pages', 'author_id']]
books_final

,isbn10,title,description,language,published_date,publisher_id,num_pages,author_id
0,439358078,Harry Potter and the Order of the Phoenix (Har...,A wizard hones his skills while fighting evil.,eng,8/21/2011,0,870.0,NaN
2,61122416,The Alchemist,"In this fable, a Spanish shepherd boy ventures...",eng,6/16/2013,284,197.0,426.0
56,385486804,Into the Wild,A man's obsession with the wilderness ends in ...,eng,12/7/2014,361,207.0,566.0
155,385494785,Into Thin Air: A Personal Account of the Mount...,A chronicle of the 1996 Mount Everest expediti...,eng,11/1/2015,361,368.0,566.0
160,307279464,A Walk in the Woods: Rediscovering America on ...,A journey to rediscover America by trekking th...,eng,10/11/2015,361,397.0,12.0
161,743270754,Team of Rivals: The Political Genius of Abraha...,The political genius of Abraham Lincoln.,eng,11/11/2012,141,916.0,682.0
182,316346624,The Tipping Point: How Little Things Can Make ...,How and why certain products and ideas become ...,eng,7/3/2011,180,301.0,664.0
273,316010669,Blink: The Power of Thinking Without Thinking,Instinct in the workings of the mind.,eng,1/3/2010,180,296.0,664.0
277,316168815,The Lovely Bones,A girl looks down from heaven as she describes...,en-US,1/3/2010,180,328.0,218.0
286,60838655,A People's History of the United States,"An account from the point of view of women, Af...",eng,2/14/2010,106,729.0,832.0


In [9]:
# Creating our engine for PostGRES database
rds_connection_string = "postgres:vfr4MKO)@localhost:5432/books_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
# Confirming our tables in the Books_DB
engine.table_names()

In [30]:
# Loading our Publisher data into the publisher table
publisher_rename.to_sql(name='publisher', con=engine, if_exists='append', index=False)

In [32]:
# Loading our author data into the author table
author_rename.to_sql(name='author', con=engine, if_exists='append', index=False)

In [42]:
# Loading our books data into the books table
books_final.to_sql(name='book', con=engine, if_exists='append', index=False)

In [49]:
# Loading our bestseller data into the bestseller table
best_sellers_df.to_sql(name='bestseller_list', con=engine, if_exists='append', index=False)

In [50]:
# Loading our reviews data into the reviews table
df_review.to_sql(name='review', con=engine, if_exists='append', index=False)

In [53]:
# Loading our sales data into the sales table
df_sales.to_sql(name='sales', con=engine, if_exists='append', index=False)

In [14]:
conn = engine.connect()

# Using the Database

Finding all books published by HarperCollins.

In [17]:
# Find books with the same publisher
books_publisher = pd.read_sql("select * from book join publisher on book.publisher_id = publisher.id where publisher_name = 'HarperCollins'", conn)
books_publisher

,isbn10,title,description,language,published_date,publisher_id,num_pages,author_id,id,publisher_name
0,61122416,The Alchemist,"In this fable, a Spanish shepherd boy ventures...",eng,2013-06-16,284,197,426,284,HarperCollins


Finding all books with non-sequential runs on the Science bestsellers list.

In [20]:
# Find books on the 'Science' bestseller list with non-sequential runs
science_bestsellers = pd.read_sql("select book.title  from book join bestseller_list on book.isbn10 = bestseller_list.isbn10 WHERE list_name = 'Science' GROUP BY bestseller_list.isbn10, book.title HAVING COUNT(*) > 1", conn)
science_bestsellers

,title
0,A Brief History of Time
